# Scraping MoonBoard Problems
Scrapes MoonBoard problems from the MoonBoard site using an automated clicking routine defined via Selenium.

In [ ]:
import shutil

from moonboard_helper import *

In [ ]:
# Load credentials
with open('./credentials.txt') as f:
    flines = f.readlines()

cred_dict = {s.split('-')[0].strip() : s.split('-')[1].strip() for s in flines}
print(cred_dict)

In [ ]:
username = cred_dict['username']
password = cred_dict['password']
driver_path = cred_dict['driver_path']
save_path = cred_dict['save_path']
save_path_holds = cred_dict['save_path_holds']
save_path_failed = cred_dict['save_path_failed']

moonboard_url = 'https://moonboard.com/'

## Preliminary Scraping (Metadata)

In [ ]:
# Load browser and login to MoonBoard
browser = load_browser(driver_path)
loginMoonBoard(browser, moonboard_url, username, password)
time.sleep(2)

In [ ]:
# Get problems view
click_view_problems(browser)
click_holdsetup(browser)

In [ ]:
# Process all pages
if not os.path.exists(save_path):
    problems_dict = process_all_pages(browser, save_path)
    save_pickle(problems_dict, save_path)
else:
    problems_dict = load_pickle(save_path)

In [ ]:
# Number of scraped problems
print('Number of problems:', len(problems_dict))

## Secondary Scraping (Problems)

In [ ]:
# Copy problem dict
if not os.path.exists(save_path_holds):
    shutil.copyfile(save_path, save_path_holds)

holds_dict = load_pickle(save_path_holds)

In [ ]:
# Failed uids
if not os.path.exists(save_path_failed):
    print('Creating failed uids dictionary...')
    failed_uids_dict = {}
    save_pickle(failed_uids_dict, save_path_failed)
else:
    print('Loading failed uids dictionary...')
    failed_uids_dict = load_pickle(save_path_failed)
    print('Number of failed Uids:', len(failed_uids_dict))

In [ ]:
# Scrape specific problems
holds_dict, failed_uids_dict = scrape_problems(
    browser, 
    holds_dict, 
    save_path_holds, 
    failed_uids_dict, 
    save_path_failed
)

In [ ]:
# Close browser
browser.close()